<a href="https://colab.research.google.com/github/Vinayak-Sharma12/Types-of-Vector-Stores-or-Databases/blob/main/Hybrid_Search_in_PineCone(PDF).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --quiet  pinecone pinecone-text langchain_community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.1 MB/s eta 0:00:00


In [5]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 2.7 MB/s eta 0:00:00


## **DOCUMENT LOADER**

In [6]:
from langchain.document_loaders import PyPDFLoader
loader=PyPDFLoader("/content/ncert_agriculture.pdf")
docs=loader.load()

In [7]:
len(docs)

10

## **CREATE CLIENT**

In [8]:
from pinecone import Pinecone
pc=Pinecone(api_key=api_key)

## **CREATE INDEX**

In [9]:
import os
from pinecone import Pinecone, ServerlessSpec

index_name = "pinecone-hybrid-search"

#Create index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # dimensionality of dense model
        metric="dotproduct",  # sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)

## **DEFINE EMBEDDING MODEL**

In [10]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/tmp/ipython-input-3588808441.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## **DEFINE SPARSE VECTOR MODEL(BM25Encoder)**

In [11]:
from pinecone_text.sparse import BM25Encoder
bm25_encoder = BM25Encoder().default()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## **CREATE HYBRID SEARCH RETRIEVER**

In [12]:
from langchain.retrievers import PineconeHybridSearchRetriever
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

## **ADD DOCUMENS TO THE RETRIEVER**

In [15]:
retriever.add_texts([doc.page_content for doc in docs])

  0%|          | 0/1 [00:00<?, ?it/s]

## **QUERY THE RETRIEVER**

In [16]:
result=retriever.invoke("Farming Techniques")

In [17]:
print(result[0].page_content)

24 resources And development
labour. Some of the operations involved are ploughing, 
sowing, irrigation, weeding and harvesting. The outputs 
from the system include crops, wool, dairy and poultry 
products.
typeS oF Farming
Farming is practised in various ways across the world. 
Depending upon the geographical conditions, demand 
of produce, labour and level of technology, farming can 
be classified into two main types. These are subsistence 
farming and commercial farming.
Subsistence Farming
This type of farming is practised to meet the needs of the 
farmer’s family. Traditionally, low levels of technology 
and household labour are used to produce on small 
output. Subsistence farming can be further classified 
as intensive subsistence and primitive subsistence 
farming.
In intensive subsistence agriculture the farmer 
cultivates a small plot of land using simple tools and 
more labour. Climate with large number of days with 
sunshine and fertile soils permit growing of more than 
o